# Интерфейсы scikit-learn

In [85]:
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [86]:
import yaml

with open('C:/Users/Zver/Desktop/uni/AIF/Practice/spbu-ai-fundamentals-main/practicum_3/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

## Estimator

Для примера построим простой estimator, который в перспективе будет вычитать из признаков их среднее значение и после сдвигать признаки на заранее заданную константу

In [87]:
class SubtractMeanAndShiftEstimator(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

In [88]:
m = SubtractMeanAndShiftEstimator(shift=3)

Метод `get_params()` реализован в `BaseEstimator`, и мы можем сразу использовать его для получения гиперпараметров модели. Это возможно, так как единственный гиперпараметр `shift` был передан как явное ключевое слово в контрукторе

Обратите внимание, что соответствующий аттрибут класса должен совпадать с ключевым словом: `self.shift = shift`

In [89]:
m.get_params()

{'shift': 3}

Аналогично мы можем использовать `set_params()` для задания значений гиперпараметров. Этот метод пригодится при поиске оптимальных значений гиперпараметров

In [90]:
m.set_params(shift=5)
m.get_params()

{'shift': 5}

In [91]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [ 0, -8],
    [ 2, 10],
    [ 1,  1],
])
m.fit(X, y)

SubtractMeanAndShiftEstimator(shift=5)

В sklearn есть класс sklearn.base.OutlierMixin, который позволяет реализовывать кастомные классы для определения выбросов.
Он добавляет:
- атрибут _estimator_type, по умолчанию outlier_detector
- fit_predict.

Метод fit() работает в формате без учителя, predict же должен классифицировать данные на аутлаеры (возвращать для них -1) и обыычные данные (возвращать 1). Для классификации используется отсечка по порогу предсказаний, полученных внутренним.
Во встроенных методах функция оценки доступна с помощью метода `score_samples`, в то время как порог можно задать параметром `contamination`. 
Например, для гауссовских данных можно использовать sklearn.covariance.EllipticEnvelope.

**Задание**: Создайте свой эстиматор с использованием sklearn.base.OutlierMixin, который будет определять выбросы на основе интерквартильного размаха. 
Он должен возвращать один столбец с 1 и -1, а также позволять задавать порог для квантиля, определяющего размах. Не забудьте, что он должен быть двухсторонним.
Ваш эстиматор должен работать и для датафреймов, и для numpy массивов.

In [92]:
from sklearn.base import BaseEstimator, OutlierMixin
class MyEstimator(BaseEstimator, OutlierMixin):
     def __init__(self,factor=1.5):
        self.factor = factor
     def fit(self, X, y=None):
         self.is_fitted_ = True
         X = pd.DataFrame(X.copy())
         q1 = X.quantile(0.35)
         q3 = X.quantile(0.55)
         iqr = q3 - q1
         self._lower_bound = q1 - (self.factor * iqr)
         self._upper_bound = q3 + (self.factor * iqr)
         print(X.min(axis=0))
         return self
         
     def predict(self, X):
         temp = np.ones(len(X))
         temp[(X > self._lower_bound.values).all(axis=1) & (X < self._upper_bound.values).all(axis=1)] = -1
         return temp


In [93]:
estimator = MyEstimator()
true_cov = np.array([[.8, .3],
                     [.3, .4]])
X = np.random.RandomState(42).multivariate_normal(mean=[0, 0],
                                                 cov=true_cov,
                                                 size=500)
e = estimator.fit(X)
#pred = estimator.predict(X)
e._lower_bound

0   -2.918998
1   -1.876028
dtype: float64


0   -0.934051
1   -0.820153
dtype: float64

In [94]:
e._upper_bound

0    0.712770
1    0.677671
dtype: float64

In [95]:
e.predict(X)

array([-1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
        1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,
       -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,
        1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
       -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
        1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1.,
        1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,
        1., -1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,
        1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
        1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,
       -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
       -1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
       -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1

## Predictor

Рассмотрим тот же класс, но добавим к нему методы `predict()` и `score()`

In [96]:
class SubtractMeanAndShiftPredictor(BaseEstimator):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def predict(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

    def score(self, X: NDArray, y: NDArray) -> float:
        return r2_score(y, self.predict(X))  # R2 \in (-\infty; 1] is the coefficient of determination

Так как мы специально добавили небольшое отклонение в y, наш R2 чуть меньше 1

In [97]:
model = SubtractMeanAndShiftPredictor(shift=1)
model.fit(X)
model.predict(X)
model.score(X, y)

ValueError: Found input variables with inconsistent numbers of samples: [3, 500]

## Transformer

Рассмотрим тот же класс, но добавим к нему метод `transform()`

In [98]:
class SubtractMeanAndShiftTransformer(BaseEstimator, OneToOneFeatureMixin, TransformerMixin):
    def __init__(self, shift=0.):
        self.shift: float = shift
        self.means_: NDArray = None  # we add a trailing underscore for parameters which will be learnt in fit()

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def transform(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X -  e @ self.means_.reshape(-1, 1).T + self.shift

In [99]:
t = SubtractMeanAndShiftTransformer(shift=5)
t.fit(X)
t.transform(X)

array([[4.61391607, 4.69717723],
       [4.09987339, 5.34409129],
       [5.26760315, 4.99381043],
       [3.46939606, 4.58731335],
       [5.29161135, 5.43772751],
       [5.5191865 , 4.99991932],
       [5.24389177, 4.04919283],
       [6.63155718, 5.5416523 ],
       [5.81384802, 5.59050612],
       [6.12193465, 4.79715004],
       [3.79685518, 4.21133018],
       [5.28182267, 4.34062904],
       [5.45600327, 5.28614781],
       [5.91904706, 5.68091218],
       [5.59757666, 5.13846277],
       [5.10346764, 6.06393206],
       [5.26709066, 4.53645106],
       [4.58232403, 4.07943612],
       [5.28320388, 4.04448556],
       [6.11345348, 5.68570059],
       [4.33352263, 4.71896187],
       [5.18066387, 4.91011528],
       [6.45502286, 5.35972974],
       [5.1651667 , 5.65563936],
       [5.12136006, 4.06699938],
       [4.82007666, 4.67052659],
       [5.45491051, 5.56348222],
       [3.90528209, 4.91152698],
       [5.80778018, 5.24123741],
       [4.49971111, 5.25422289],
       [5.

Так как мы добавили `TransformerMixin`, мы можем использовать метод `fit_transform()`, не реализуя его явно

In [100]:
t.fit_transform(X)

array([[4.61391607, 4.69717723],
       [4.09987339, 5.34409129],
       [5.26760315, 4.99381043],
       [3.46939606, 4.58731335],
       [5.29161135, 5.43772751],
       [5.5191865 , 4.99991932],
       [5.24389177, 4.04919283],
       [6.63155718, 5.5416523 ],
       [5.81384802, 5.59050612],
       [6.12193465, 4.79715004],
       [3.79685518, 4.21133018],
       [5.28182267, 4.34062904],
       [5.45600327, 5.28614781],
       [5.91904706, 5.68091218],
       [5.59757666, 5.13846277],
       [5.10346764, 6.06393206],
       [5.26709066, 4.53645106],
       [4.58232403, 4.07943612],
       [5.28320388, 4.04448556],
       [6.11345348, 5.68570059],
       [4.33352263, 4.71896187],
       [5.18066387, 4.91011528],
       [6.45502286, 5.35972974],
       [5.1651667 , 5.65563936],
       [5.12136006, 4.06699938],
       [4.82007666, 4.67052659],
       [5.45491051, 5.56348222],
       [3.90528209, 4.91152698],
       [5.80778018, 5.24123741],
       [4.49971111, 5.25422289],
       [5.

Аналогично мы можем использовать метод `get_feature_names_out()`, так как мы добавили `OneToOneFeatureMixin`

In [101]:
t.get_feature_names_out(input_features=['x', 'y'])

NotFittedError: This SubtractMeanAndShiftTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [102]:
Используем датасет с домами как пример. Вспомним, что мы делали в прошлый раз, и попробуем заполнить пропущенные значения в некоторых числовызх столбцах.
Для этого используем трансформер по столбцам. 

SyntaxError: invalid syntax (3535100906.py, line 1)

In [103]:
df = pd.read_csv(cfg['house_pricing']['train_dataset'])
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


По умолчанию, только указанные столбцы трансформируются и возвращаются (remainder=`drop`). Мы же сделаем так, чтобы все остальные столбцы тоже возвращались, просто с ними бы ничего не делалось. 

In [104]:
ct = ColumnTransformer(
    [('mean_impute', SimpleImputer(strategy='mean'), ['SalePrice', 'LotArea', 'WoodDeckSF',  'MasVnrArea'])], 
    remainder="passthrough")

ct.fit(df)

C:\Users\Zver\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('mean_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea'])])

In [105]:
ct.transform(df)

array([[208500.0, 8450.0, 0.0, ..., 2008, 'WD', 'Normal'],
       [181500.0, 9600.0, 298.0, ..., 2007, 'WD', 'Normal'],
       [223500.0, 11250.0, 0.0, ..., 2008, 'WD', 'Normal'],
       ...,
       [266500.0, 9042.0, 0.0, ..., 2010, 'WD', 'Normal'],
       [142125.0, 9717.0, 366.0, ..., 2010, 'WD', 'Normal'],
       [147500.0, 9937.0, 736.0, ..., 2008, 'WD', 'Normal']], dtype=object)

In [106]:
ct.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('mean_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea'])])

Если у датасета появятсся столбцы, которые не были представлены во время fit (даже среди тех, что не трансформировались), то они будут выкинуты на этапе transform. 

In [107]:
df["temp"] = 0
ct.transform(df)

,mean_impute__SalePrice,mean_impute__LotArea,mean_impute__WoodDeckSF,mean_impute__MasVnrArea,remainder__Id,remainder__MSSubClass,remainder__MSZoning,remainder__LotFrontage,remainder__Street,remainder__Alley,...,remainder__ScreenPorch,remainder__PoolArea,remainder__PoolQC,remainder__Fence,remainder__MiscFeature,remainder__MiscVal,remainder__MoSold,remainder__YrSold,remainder__SaleType,remainder__SaleCondition
0,208500.0,8450.0,0.0,196.0,1,60,RL,65.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,181500.0,9600.0,298.0,0.0,2,20,RL,80.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,223500.0,11250.0,0.0,162.0,3,60,RL,68.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,140000.0,9550.0,0.0,0.0,4,70,RL,60.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,250000.0,14260.0,192.0,350.0,5,60,RL,84.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,175000.0,7917.0,0.0,0.0,1456,60,RL,62.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,210000.0,13175.0,349.0,119.0,1457,20,RL,85.0,Pave,NaN,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,266500.0,9042.0,0.0,0.0,1458,70,RL,66.0,Pave,NaN,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,142125.0,9717.0,366.0,0.0,1459,20,RL,68.0,Pave,NaN,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


**Задание**: Перейдите к медианному заполнению пропусков. Проверьте, что результаты, полученные с помощью трансформации, соответствуют преобразованию напрямую.

In [108]:
columns_to_impute = ['SalePrice', 'LotArea', 'WoodDeckSF', 'MasVnrArea']

ct = ColumnTransformer(
    [('median_impute', SimpleImputer(strategy='median'), columns_to_impute)],
    remainder="passthrough",
    verbose_feature_names_out=False
)

ct.fit(df)
df_transformed = ct.transform(df)

feature_names = ct.get_feature_names_out()
df_transformed = pd.DataFrame(df_transformed, columns=feature_names)

print("\nDataFrame после трансформации ColumnTransformer (медиана):")
print(df_transformed.head())

df_direct = df.copy()

for col in columns_to_impute:
    median_value = df[col].median()
    df_direct[col] = df_direct[col].fillna(median_value)

print("\nDataFrame после прямого заполнения медианой:")
print(df_direct.head())

for col in columns_to_impute:
    try:
        transformed_values = df_transformed[col].head(5).astype(float).values
        direct_values = df_direct[col].head(5).astype(float).values
    except ValueError as e:
        print(f"Ошибка при преобразовании столбца '{col}' в числовой тип: {e}")
        continue

    if np.allclose(transformed_values, direct_values):
        print(f"\nСтолбец '{col}': Значения, полученные с помощью ColumnTransformer, совпадают со значениями после прямого заполнения.")
    else:
        print(f"\nСтолбец '{col}': Значения не совпадают.")
        print("Transformed:", transformed_values)
        print("Direct:", direct_values)

df["temp"] = 0
try:
  df_transformed_with_temp = ct.transform(df)
  feature_names_with_temp = ct.get_feature_names_out()
  df_transformed_with_temp = pd.DataFrame(df_transformed_with_temp, columns=feature_names_with_temp) # Возвращаем DataFrame
  print("\nDataFrame после transform с новым столбцом:")
  print(df_transformed_with_temp.head())

  if "temp" in df_transformed_with_temp.columns:
     print("\nНовый столбец 'temp' успешно добавлен в transformed DataFrame.")
  else:
     print("\nНовый столбец 'temp' не был добавлен.")
except ValueError as e:
    print(f"\nОшибка при transform с новым столбцом: {e}")


DataFrame после трансформации ColumnTransformer (медиана):
  SalePrice  LotArea WoodDeckSF MasVnrArea Id MSSubClass MSZoning LotFrontage  \
0  208500.0   8450.0        0.0      196.0  1         60       RL        65.0   
1  181500.0   9600.0      298.0        0.0  2         20       RL        80.0   
2  223500.0  11250.0        0.0      162.0  3         60       RL        68.0   
3  140000.0   9550.0        0.0        0.0  4         70       RL        60.0   
4  250000.0  14260.0      192.0      350.0  5         60       RL        84.0   

  Street Alley  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold YrSold  \
0   Pave   NaN  ...        0    NaN   NaN         NaN       0      2   2008   
1   Pave   NaN  ...        0    NaN   NaN         NaN       0      5   2007   
2   Pave   NaN  ...        0    NaN   NaN         NaN       0      9   2008   
3   Pave   NaN  ...        0    NaN   NaN         NaN       0      2   2006   
4   Pave   NaN  ...        0    NaN   NaN         NaN     

**Задание**: Добавьте еще нормализатор для LotFrontage, LotArea и запустите в ColumnTransformer. Обучите его и примените.

In [109]:
from sklearn.preprocessing import MinMaxScaler
columns_to_impute = ['SalePrice', 'WoodDeckSF', 'MasVnrArea']
columns_to_normalize = ['LotFrontage', 'LotArea']

ct = ColumnTransformer(
    [
        ('median_impute', SimpleImputer(strategy='median'), columns_to_impute),
        ('normalize', MinMaxScaler(), columns_to_normalize)
    ],
    remainder="passthrough",
    verbose_feature_names_out=False
)

ct.fit(df)
df_transformed = ct.transform(df)

feature_names = ct.get_feature_names_out()
df_transformed = pd.DataFrame(df_transformed, columns=feature_names)

print("\nDataFrame после трансформации ColumnTransformer (медиана + нормализация):")
print(df_transformed.head())

print("\nПроверка нормализации:")
for col in columns_to_normalize:
    print(f"Столбец '{col}': min = {df_transformed[col].min()}, max = {df_transformed[col].max()}")

print("\nПример использования: Первые 5 строк столбца 'SalePrice' после заполнения медианой:")
print(df_transformed['SalePrice'].head())

df["temp"] = 0
try:
  df_transformed_with_temp = ct.transform(df)
  feature_names_with_temp = ct.get_feature_names_out()
  df_transformed_with_temp = pd.DataFrame(df_transformed_with_temp, columns=feature_names_with_temp) # Возвращаем DataFrame
  print("\nDataFrame после transform с новым столбцом:")
  print(df_transformed_with_temp.head())

  if "temp" in df_transformed_with_temp.columns:
     print("\nНовый столбец 'temp' успешно добавлен в transformed DataFrame.")
  else:
     print("\nНовый столбец 'temp' не был добавлен.")
except ValueError as e:
    print(f"\nОшибка при transform с новым столбцом: {e}")


DataFrame после трансформации ColumnTransformer (медиана + нормализация):
  SalePrice WoodDeckSF MasVnrArea LotFrontage   LotArea Id MSSubClass  \
0  208500.0        0.0      196.0    0.150685   0.03342  1         60   
1  181500.0      298.0        0.0    0.202055  0.038795  2         20   
2  223500.0        0.0      162.0    0.160959  0.046507  3         60   
3  140000.0        0.0        0.0    0.133562  0.038561  4         70   
4  250000.0      192.0      350.0    0.215753  0.060576  5         60   

  MSZoning Street Alley  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0       RL   Pave   NaN  ...        0    NaN   NaN         NaN       0      2   
1       RL   Pave   NaN  ...        0    NaN   NaN         NaN       0      5   
2       RL   Pave   NaN  ...        0    NaN   NaN         NaN       0      9   
3       RL   Pave   NaN  ...        0    NaN   NaN         NaN       0      2   
4       RL   Pave   NaN  ...        0    NaN   NaN         NaN       0     12   


Sklearn располагает большим количеством встроенных трансформеров. Соответствующие трансформеры есть и для категориальных фичей (более подробно рассмотрим этот тип чуть позже). Например, известное нам бинарное кодирование можно проводить с помощью OneHotEncoder()

In [110]:
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(
    transformers=[
        ('median_impute', SimpleImputer(strategy='mean'), ['SalePrice', 'LotArea', 'WoodDeckSF',  'MasVnrArea']),
        ("one_hot_encode", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["MSZoning", "SaleType", "SaleCondition"]),
    ], 
    remainder="passthrough")

ct.fit(df)

C:\Users\Zver\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('median_impute', SimpleImputer(),
                                 ['SalePrice', 'LotArea', 'WoodDeckSF',
                                  'MasVnrArea']),
                                ('one_hot_encode',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['MSZoning', 'SaleType', 'SaleCondition'])])

In [111]:
ct.set_output(transform='pandas')
ct.transform(df)

,median_impute__SalePrice,median_impute__LotArea,median_impute__WoodDeckSF,median_impute__MasVnrArea,one_hot_encode__MSZoning_C (all),one_hot_encode__MSZoning_FV,one_hot_encode__MSZoning_RH,one_hot_encode__MSZoning_RL,one_hot_encode__MSZoning_RM,one_hot_encode__SaleType_COD,...,remainder__3SsnPorch,remainder__ScreenPorch,remainder__PoolArea,remainder__PoolQC,remainder__Fence,remainder__MiscFeature,remainder__MiscVal,remainder__MoSold,remainder__YrSold,remainder__temp
0,208500.0,8450.0,0.0,196.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,2,2008,0
1,181500.0,9600.0,298.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,5,2007,0
2,223500.0,11250.0,0.0,162.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,9,2008,0
3,140000.0,9550.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,2,2006,0
4,250000.0,14260.0,192.0,350.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,12,2008,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,175000.0,7917.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,8,2007,0
1456,210000.0,13175.0,349.0,119.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,MnPrv,NaN,0,2,2010,0
1457,266500.0,9042.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,GdPrv,Shed,2500,5,2010,0
1458,142125.0,9717.0,366.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,NaN,NaN,NaN,0,4,2010,0


Для выбора столбцов можно создавать make_selector, например, по выбору численных и категориальных значений. 

**Доп. задание**. Сделайте трансформер для OneHotEncoder на основе make_selector так, чтобы выбирать все нечисловые столбцы. Сколько столбцов получается после трансформации?

In [114]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
import numpy as np

#!!! Здесь я начал писать комментарии чтобы не запутаться, если нужно, я уберу и обновлю.

# 1. Создаем ColumnTransformer
ct = ColumnTransformer(
    [
        ('onehot_encode', OneHotEncoder(handle_unknown='ignore'), selector(dtype_exclude=np.number))
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

# 2. Обучение и трансформация
ct.fit(df)
df_transformed = ct.transform(df)

# Преобразуем разреженную матрицу в плотный массив и создаем DataFrame
df_transformed = pd.DataFrame(df_transformed.toarray())

print("\nDataFrame после трансформации ColumnTransformer (OneHotEncoding для нечисловых столбцов):")
print(df_transformed.head())

# 3. Вывод количества столбцов после трансформации
num_columns_after = df_transformed.shape[1]
print(f"\nКоличество столбцов после трансформации: {num_columns_after}")


DataFrame после трансформации ColumnTransformer (OneHotEncoding для нечисловых столбцов):
   0    1    2    3    4    5    6    7    8    9    ...  257  258  259  260  \
0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  1.0   
1  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  1.0   
2  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  1.0   
3  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  1.0   
4  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  1.0   

   261  262  263  264  265  266  
0  0.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  0.0  0.0  0.0  1.0  0.0  
2  0.0  0.0  0.0  0.0  1.0  0.0  
3  1.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  1.0  0.0  

[5 rows x 267 columns]

Количество столбцов после трансформации: 267


## Pipelines

С помощью Pipeline мы можем производить последовательную обработку данных и выполнять предсказание в конце

In [115]:
X = np.array([
    [1, 10],
    [3, 30],
    [2, 20],
])
y = np.array([
    [0],
    [2],
    [1],
])

pipeline = Pipeline([
    ("shifter", SubtractMeanAndShiftTransformer(shift=5)),
    ("regressor", LinearRegression()),
])
...
pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)

[[-4.4408921e-16]
 [ 2.0000000e+00]
 [ 1.0000000e+00]]


Pipeline хранит последовательные Estimators в аттрибуте `steps`

In [116]:
pipeline.steps

[('shifter', SubtractMeanAndShiftTransformer(shift=5)),
 ('regressor', LinearRegression())]

Перейти к объекту i-го Estimator можно напрямую через `pipeline[i]`:

In [117]:
pipeline[0]

SubtractMeanAndShiftTransformer(shift=5)

In [118]:
pipeline[1].coef_

array([[0.00990099, 0.0990099 ]])

Так как Pipeline сам является Estimator, мы можем увидеть список его параметров:

In [119]:
pipeline.get_params()

{'memory': None,
 'steps': [('shifter', SubtractMeanAndShiftTransformer(shift=5)),
  ('regressor', LinearRegression())],
 'transform_input': None,
 'verbose': False,
 'shifter': SubtractMeanAndShiftTransformer(shift=5),
 'regressor': LinearRegression(),
 'shifter__shift': 5,
 'regressor__copy_X': True,
 'regressor__fit_intercept': True,
 'regressor__n_jobs': None,
 'regressor__positive': False}

Видно, параметры промежуточных Estimator указаны как `<estimator>__<parameter>`. Следовательно, мы можем изменить параметры любого промежуточного Estimator:

In [120]:
pipeline.set_params(shifter__shift=10)
pipeline.get_params()

{'memory': None,
 'steps': [('shifter', SubtractMeanAndShiftTransformer(shift=10)),
  ('regressor', LinearRegression())],
 'transform_input': None,
 'verbose': False,
 'shifter': SubtractMeanAndShiftTransformer(shift=10),
 'regressor': LinearRegression(),
 'shifter__shift': 10,
 'regressor__copy_X': True,
 'regressor__fit_intercept': True,
 'regressor__n_jobs': None,
 'regressor__positive': False}

In [121]:
**Задание**: Создайте пайплайн по преобразованию чсиленных столбцов, содержащий импьютер и скейлер.

SyntaxError: invalid syntax (874014090.py, line 1)

In [122]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
import numpy as np

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler())])                 

numerical_cols = selector(dtype_include=np.number)

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_cols)
    ],
    remainder='passthrough'
)

X = pd.DataFrame({
    'num1': [1, 2, np.nan, 4, 5],
    'num2': [5, np.nan, 7, 8, 9],
    'cat1': ['A', 'B', 'A', 'C', 'B']
})

print("\nИсходные данные для демонстрации:")
print(X)

preprocessor.fit(X)
X_transformed = preprocessor.transform(X)

print("\nПреобразованные данные:")
print(X_transformed)

print("\nШаги в numerical_pipeline:")
print(numerical_pipeline.steps)

print("\nПараметры numerical_pipeline:")
print(numerical_pipeline.get_params())

numerical_pipeline.set_params(imputer__strategy='mean')

print("\nПараметры numerical_pipeline после изменения:")
print(numerical_pipeline.get_params())


Исходные данные для демонстрации:
   num1  num2 cat1
0   1.0   5.0    A
1   2.0   NaN    B
2   NaN   7.0    A
3   4.0   8.0    C
4   5.0   9.0    B

Преобразованные данные:
[[-1.414213562373095 -1.7336902313221405 'A']
 [-0.7071067811865475 0.15075567228888193 'B']
 [0.0 -0.22613350843332256 'A']
 [0.7071067811865475 0.5276448530110864 'C']
 [1.414213562373095 1.2814232144554953 'B']]

Шаги в numerical_pipeline:
[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]

Параметры numerical_pipeline:
{'memory': None, 'steps': [('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())], 'transform_input': None, 'verbose': False, 'imputer': SimpleImputer(strategy='median'), 'scaler': StandardScaler(), 'imputer__add_indicator': False, 'imputer__copy': True, 'imputer__fill_value': None, 'imputer__keep_empty_features': False, 'imputer__missing_values': nan, 'imputer__strategy': 'median', 'scaler__copy': True, 'scaler__with_mean': True, 'scaler__with_std

**Задание**: Создайте новый трансформер, который для категориальных столбцов будет заполнять пропущенные значения наиболее часто встречаюмшщимся или новой категорией (сделайте параметром). 

In [123]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.utils.validation import check_is_fitted
import numpy as np

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Заполняет пропущенные значения в категориальных столбцах.

    Parameters
    ----------
    strategy : str, default='most_frequent'
        Стратегия заполнения пропусков. Может быть 'most_frequent' для заполнения
        наиболее часто встречающимся значением или 'new_category' для заполнения
        новой категорией.
    fill_value : str, default='missing'
        Значение, используемое для заполнения пропусков, если strategy='new_category'.

    Attributes
    ----------
    imputer_ : SimpleImputer
        Внутренний SimpleImputer для заполнения пропусков наиболее часто встречающимся значением.
    """

    def __init__(self, strategy='most_frequent', fill_value='missing'):
        self.strategy = strategy
        self.fill_value = fill_value
        self.fitted_ = False

    def fit(self, X, y=None):
        """
        Обучает импьютер.

        Parameters
        ----------
        X : pandas DataFrame, shape (n_samples, n_features)
            Входные данные.

        y : Ignored
            Не используется, присутствует для совместимости с API scikit-learn.

        Returns
        -------
        self : object
            Возвращает обученный экземпляр класса.
        """

        if self.strategy not in ['most_frequent', 'new_category']:
            raise ValueError("Strategy must be 'most_frequent' or 'new_category'")

        if self.strategy == 'most_frequent':
            self.imputer_ = SimpleImputer(strategy='most_frequent')
            self.imputer_.fit(X)
        self.fitted_ = True
        return self

    def transform(self, X):
        """
        Преобразует данные, заполняя пропущенные значения.

        Parameters
        ----------
        X : pandas DataFrame, shape (n_samples, n_features)
            Входные данные.

        Returns
        -------
        X_transformed : pandas DataFrame, shape (n_samples, n_features)
            Преобразованные данные.
        """
        if not self.fitted_:
            raise NotFittedError("This CategoricalImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        X = X.copy()

        if self.strategy == 'most_frequent':
            X = pd.DataFrame(self.imputer_.transform(X), columns=X.columns, index=X.index)
        else: 
            self.strategy == 'new_category'
            X = X.fillna(self.fill_value)

        return X

print("Исходный DataFrame:")
print(df.head())

# 1. Создаем экземпляр трансформера
categorical_imputer = CategoricalImputer(strategy='new_category', fill_value='missing_value')

# 2. Выбираем категориальные столбцы
categorical_cols = df.select_dtypes(exclude=np.number).columns

# 3. Обучаем трансформер
categorical_imputer.fit(df[categorical_cols])

# 4. Применяем трансформер
df[categorical_cols] = categorical_imputer.transform(df[categorical_cols])

print("\nDataFrame после заполнения пропусков в категориальных столбцах:")
print(df.head())

# 5. Пример с ColumnTransformer
ct = ColumnTransformer(
    [
        ('cat_impute', CategoricalImputer(strategy='most_frequent'), selector(dtype_exclude=np.number))
    ],
    remainder='passthrough'
)

# Обучаем и применяем ColumnTransformer
ct.fit(df)
df_transformed = ct.transform(df)

print("\nDataFrame после ColumnTransformer:")
print(pd.DataFrame(df_transformed).head())

Исходный DataFrame:
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolQC Fence MiscFeature MiscVal MoSold YrSold  \
0         Lvl    AllPub  ...    NaN   NaN         NaN       0      2   2008   
1         Lvl    AllPub  ...    NaN   NaN         NaN       0      5   2007   
2         Lvl    AllPub  ...    NaN   NaN         NaN       0      9   2008   
3         Lvl    AllPub  ...    NaN   NaN         NaN       0      2   2006   
4         Lvl    AllPub  ...    NaN   NaN         NaN       0     12   2008   

  SaleType  SaleCondition  SalePrice  temp  
0       WD     

**Задание**: Создайте пайплайн по преобразованию категориальных столбцов, содержащий ваш импьютер и OneHotEncoder.

In [124]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import numpy as np

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Заполняет пропущенные значения в категориальных столбцах.

    Parameters
    ----------
    strategy : str, default='most_frequent'
        Стратегия заполнения пропусков. Может быть 'most_frequent' для заполнения
        наиболее часто встречающимся значением или 'new_category' для заполнения
        новой категорией.
    fill_value : str, default='missing'
        Значение, используемое для заполнения пропусков, если strategy='new_category'.

    Attributes
    ----------
    imputer_ : SimpleImputer
        Внутренний SimpleImputer для заполнения пропусков наиболее часто встречающимся значением.
    """

    def __init__(self, strategy='most_frequent', fill_value='missing'):
        self.strategy = strategy
        self.fill_value = fill_value
        self.fitted_ = False

    def fit(self, X, y=None):
        """
        Обучает импьютер.

        Parameters
        ----------
        X : pandas DataFrame, shape (n_samples, n_features)
            Входные данные.

        y : Ignored
            Не используется, присутствует для совместимости с API scikit-learn.

        Returns
        -------
        self : object
            Возвращает обученный экземпляр класса.
        """

        if self.strategy not in ['most_frequent', 'new_category']:
            raise ValueError("Strategy must be 'most_frequent' or 'new_category'")

        if self.strategy == 'most_frequent':
            self.imputer_ = SimpleImputer(strategy='most_frequent')
            self.imputer_.fit(X)
        self.fitted_ = True
        return self

    def transform(self, X):
        """
        Преобразует данные, заполняя пропущенные значения.

        Parameters
        ----------
        X : pandas DataFrame, shape (n_samples, n_features)
            Входные данные.

        Returns
        -------
        X_transformed : pandas DataFrame, shape (n_samples, n_features)
            Преобразованные данные.
        """
        if not self.fitted_:
            raise NotFittedError("This CategoricalImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        X = X.copy()

        if self.strategy == 'most_frequent':
            X = pd.DataFrame(self.imputer_.transform(X), columns=X.columns, index=X.index)
        else:
            X = X.fillna(self.fill_value)

        return X

# 1. Создаем Pipeline для категориальных столбцов
categorical_pipeline = Pipeline([
    ('imputer', CategoricalImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))         
])

# 2. Создаем ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_pipeline, selector(dtype_exclude=np.number))  # Применяем pipeline к категориальным столбцам
    ],
    remainder='passthrough'  
)

# 3. Используем исходный DataFrame для демонстрации
X = df.copy()

print("\nИсходные данные для демонстрации:")
print(X.head())

# 4. Обучаем и трансформируем данные с помощью Pipeline
preprocessor.fit(X)
X_transformed = preprocessor.transform(X)

# Преобразуем разреженную матрицу в плотный массив NumPy и создаем DataFrame
X_transformed_dense = X_transformed.toarray()
df_transformed = pd.DataFrame(X_transformed_dense)

print("\nПреобразованные данные (DataFrame):")
print(df_transformed.head())

# 5. Вывод количества столбцов после трансформации
num_columns_after = df_transformed.shape[1]
print(f"\nКоличество столбцов после трансформации: {num_columns_after}")


Исходные данные для демонстрации:
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street          Alley  \
0   1          60       RL         65.0     8450   Pave  missing_value   
1   2          20       RL         80.0     9600   Pave  missing_value   
2   3          60       RL         68.0    11250   Pave  missing_value   
3   4          70       RL         60.0     9550   Pave  missing_value   
4   5          60       RL         84.0    14260   Pave  missing_value   

  LotShape LandContour Utilities  ...         PoolQC          Fence  \
0      Reg         Lvl    AllPub  ...  missing_value  missing_value   
1      Reg         Lvl    AllPub  ...  missing_value  missing_value   
2      IR1         Lvl    AllPub  ...  missing_value  missing_value   
3      IR1         Lvl    AllPub  ...  missing_value  missing_value   
4      IR1         Lvl    AllPub  ...  missing_value  missing_value   

     MiscFeature MiscVal MoSold YrSold SaleType  SaleCondition  SalePrice  \
0  missing_value

**Задание**: Создайте ColumnTransformer, который будет содержать в себе два вышеуказанных пайплайна.

In [125]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import numpy as np

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Заполняет пропущенные значения в категориальных столбцах.

    Parameters
    ----------
    strategy : str, default='most_frequent'
        Стратегия заполнения пропусков. Может быть 'most_frequent' для заполнения
        наиболее часто встречающимся значением или 'new_category' для заполнения
        новой категорией.
    fill_value : str, default='missing'
        Значение, используемое для заполнения пропусков, если strategy='new_category'.
    """

    def __init__(self, strategy='most_frequent', fill_value='missing'):
        self.strategy = strategy
        self.fill_value = fill_value
        self.fitted_ = False

    def fit(self, X, y=None):
        if self.strategy not in ['most_frequent', 'new_category']:
            raise ValueError("Strategy must be 'most_frequent' or 'new_category'")

        if self.strategy == 'most_frequent':
            self.imputer_ = SimpleImputer(strategy='most_frequent')
            self.imputer_.fit(X)
        self.fitted_ = True
        return self

    def transform(self, X):
        if not self.fitted_:
            raise NotFittedError("This CategoricalImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        X = X.copy()

        if self.strategy == 'most_frequent':
            X = pd.DataFrame(self.imputer_.transform(X), columns=X.columns, index=X.index)
        else:
            X = X.fillna(self.fill_value)

        return X


# 1. Создаем Pipeline для численных столбцов
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. Создаем Pipeline для категориальных столбцов
categorical_pipeline = Pipeline([
    ('imputer', CategoricalImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Создаем ColumnTransformer, объединяющий оба пайплайна
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, selector(dtype_include=np.number)),
        ('categorical', categorical_pipeline, selector(dtype_exclude=np.number))
    ],
    remainder='passthrough'
)

# 4. Применяем ColumnTransformer к данным
df_transformed = preprocessor.fit_transform(df)

# Преобразуем разреженную матрицу в плотный массив NumPy
df_transformed = df_transformed.toarray()

# 5. Преобразуем результат в DataFrame
df_transformed = pd.DataFrame(df_transformed)

# Вывод первых строк преобразованного DataFrame
print("\nПреобразованный DataFrame:")
print(df_transformed.head())

# Вывод количества столбцов после трансформации
num_columns_after = df_transformed.shape[1]
print(f"\nКоличество столбцов после трансформации: {num_columns_after}")


Преобразованный DataFrame:
        0         1         2         3         4         5         6    \
0 -1.730865  0.073375 -0.220875 -0.207142  0.651479 -0.517200  1.050994   
1 -1.728492 -0.872563  0.460320 -0.091886 -0.071836  2.179628  0.156734   
2 -1.726120  0.073375 -0.084636  0.073480  0.651479 -0.517200  0.984752   
3 -1.723747  0.309859 -0.447940 -0.096897  0.651479 -0.517200 -1.863632   
4 -1.721374  0.073375  0.641972  0.375148  1.374795 -0.517200  0.951632   

        7         8         9    ...  296  297  298  299  300  301  302  303  \
0  0.878668  0.514104  0.575425  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
1 -0.429577 -0.570750  1.171992  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
2  0.830215  0.325915  0.092907  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
3 -0.720298 -0.570750 -0.499274  ...  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
4  0.733308  1.366489  0.463568  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   

   304  305  
0  1.0  0.0  
1  1.0  0.0 

**Доп.задание**: Используйте для комбинации результатов двух отдельных трансформеров FeatureUnion

In [126]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
import numpy as np

class CategoricalImputer(BaseEstimator, TransformerMixin):
    """
    Заполняет пропущенные значения в категориальных столбцах.

    Parameters
    ----------
    strategy : str, default='most_frequent'
        Стратегия заполнения пропусков. Может быть 'most_frequent' для заполнения
        наиболее часто встречающимся значением или 'new_category' для заполнения
        новой категорией.
    fill_value : str, default='missing'
        Значение, используемое для заполнения пропусков, если strategy='new_category'.
    """

    def __init__(self, strategy='most_frequent', fill_value='missing'):
        self.strategy = strategy
        self.fill_value = fill_value
        self.fitted_ = False

    def fit(self, X, y=None):
        if self.strategy not in ['most_frequent', 'new_category']:
            raise ValueError("Strategy must be 'most_frequent' or 'new_category'")

        if self.strategy == 'most_frequent':
            self.imputer_ = SimpleImputer(strategy='most_frequent')
            self.imputer_.fit(X)
        self.fitted_ = True
        return self

    def transform(self, X):
        if not self.fitted_:
            raise NotFittedError("This CategoricalImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.")

        X = X.copy()

        if self.strategy == 'most_frequent':
            X = pd.DataFrame(self.imputer_.transform(X), columns=X.columns, index=X.index)
        else:
            X = X.fillna(self.fill_value)

        return X

# Функция для выбора числовых столбцов
def select_numeric(X):
    return X.select_dtypes(include=np.number)

# Функция для выбора категориальных столбцов
def select_categorical(X):
    return X.select_dtypes(exclude=np.number)

# 1. Создаем Pipeline для численных столбцов
numerical_pipeline = Pipeline([
    ('selector', FunctionTransformer(select_numeric)),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. Создаем Pipeline для категориальных столбцов
categorical_pipeline = Pipeline([
    ('selector', FunctionTransformer(select_categorical)),
    ('imputer', CategoricalImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Используем FeatureUnion для объединения результатов
preprocessor = FeatureUnion(
    transformer_list=[
        ('numerical', numerical_pipeline),
        ('categorical', categorical_pipeline)
    ]
)

# 4. Применяем FeatureUnion к данным
df_transformed = preprocessor.fit_transform(df)

# Преобразуем разреженную матрицу в плотный массив NumPy
df_transformed = df_transformed.toarray()

# 5. Преобразуем результат в DataFrame
df_transformed = pd.DataFrame(df_transformed)

# Вывод первых строк преобразованного DataFrame
print("\nПреобразованный DataFrame:")
print(df_transformed.head())

# Вывод количества столбцов после трансформации
num_columns_after = df_transformed.shape[1]
print(f"\nКоличество столбцов после трансформации: {num_columns_after}")


Преобразованный DataFrame:
        0         1         2         3         4         5         6    \
0 -1.730865  0.073375 -0.220875 -0.207142  0.651479 -0.517200  1.050994   
1 -1.728492 -0.872563  0.460320 -0.091886 -0.071836  2.179628  0.156734   
2 -1.726120  0.073375 -0.084636  0.073480  0.651479 -0.517200  0.984752   
3 -1.723747  0.309859 -0.447940 -0.096897  0.651479 -0.517200 -1.863632   
4 -1.721374  0.073375  0.641972  0.375148  1.374795 -0.517200  0.951632   

        7         8         9    ...  296  297  298  299  300  301  302  303  \
0  0.878668  0.514104  0.575425  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
1 -0.429577 -0.570750  1.171992  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
2  0.830215  0.325915  0.092907  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
3 -0.720298 -0.570750 -0.499274  ...  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0   
4  0.733308  1.366489  0.463568  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   

   304  305  
0  1.0  0.0  
1  1.0  0.0 